In [1]:
import jsonlines
import pdb
from tqdm import tqdm

train_file = './data/snli/snli_1.0/snli_1.0_train.jsonl'

train = []
label = []
label_map = {'contradiction':0, 'entailment':1, 'neutral':2}

with open(train_file,'r') as f:
    for item in tqdm(jsonlines.Reader(f)):
        if item['gold_label'] not in label_map:
            continue
        train.append([item['sentence1'],item['sentence2']])
        label.append(label_map[item['gold_label']])

550152it [00:04, 137330.41it/s]


In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup

from keras.utils import to_categorical
import random
import numpy as np
#from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split # 划分训练集和测试集

np.random.seed(123)
random.seed(123)
import re

Using TensorFlow backend.


In [3]:
train[0]

['A person on a horse jumps over a broken down airplane.',
 'A person is training his horse for a competition.']

In [4]:
def clean_sentences(df):
    reviews = []
    for sent in tqdm(df):
        lemma_words = []
        word = re.sub('[^a-zA-Z]',' ',sent[0])
        word = word_tokenize(word.lower())
        lemma_words.append([lemmatizer.lemmatize(i) for i in word])
        word = re.sub('[^a-zA-Z]',' ',sent[1])
        word = word_tokenize(word.lower())
        lemma_words.append([lemmatizer.lemmatize(i) for i in word])
        reviews.append(lemma_words)
    return reviews

In [5]:
train_sentences = clean_sentences(train)
len(train_sentences)

100%|██████████| 549367/549367 [01:50<00:00, 4980.72it/s]


549367

In [6]:
print(len(train_sentences),len(label))

549367 549367


In [8]:
target = to_categorical(label)
nums_classes = 3
# 拿到词典
words_dict = set()
len_max = 0
for sent in tqdm(train_sentences):
    words_dict.update(sent[0])
    words_dict.update(sent[1])
    if len_max < max(len(sent[0]),len(sent[1])):
        len_max = max(len(sent[0]),len(sent[1]))
print(len(words_dict))
print(len_max)
# split dataset
print('划分数据集')
x_train,x_val,y_train,y_val = train_test_split(train_sentences,target,test_size=0.2,stratify=target)

100%|██████████| 549367/549367 [00:01<00:00, 403942.46it/s]


28420
78
划分数据集


In [10]:
list(words_dict)[:10]

['computing',
 'predicting',
 'slinkys',
 'subaru',
 'note',
 'booty',
 'subsequently',
 'ralley',
 'dandelion',
 'chessboard']

In [63]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
# 对句子进行tokenizer操作
tokenizer = Tokenizer(num_words = len(list(words_dict)))
sentence1 = list(np.array(train_sentences)[::,0])
sentence2 = list(np.array(train_sentences)[::,0])
print('分词')
tokenizer.fit_on_texts(sentence1 + sentence2)
print('sentence to sequence')
x_train1 = list(np.array(x_train)[::,0])
x_train2 = list(np.array(x_train)[::,1])
x_val1 = list(np.array(x_val)[::,0])
x_val2 = list(np.array(x_val)[::,1])
x_train1 = tokenizer.texts_to_sequences(x_train1)
x_train2 = tokenizer.texts_to_sequences(x_train2)
x_val1 = tokenizer.texts_to_sequences(x_val1)
x_val2 = tokenizer.texts_to_sequences(x_val2)

分词
sentence to sequence


In [65]:
# padding sequence
print('padding to sequence ')
x_train1 = sequence.pad_sequences(x_train1,maxlen=len_max)
x_train2 = sequence.pad_sequences(x_train2,maxlen=len_max)
x_val1 = sequence.pad_sequences(x_val1,maxlen=len_max)
x_val2 = sequence.pad_sequences(x_val2,maxlen=len_max)

padding to sequence 


In [67]:
print(x_train1.shape,x_train2.shape,x_val1.shape,x_val2.shape)

(439493, 78) (439493, 78) (109874, 78) (109874, 78)


In [69]:
x_train1[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   1,  21,   6,   2,
         1,  23,  16,   7, 514,   1, 258,   8, 662,   5,   1, 293, 541],
      dtype=int32)

In [71]:
# 设置early stop
early_stopping = EarlyStopping(min_delta = 0.001,mode = 'max',monitor = 'val_acc',patience=2)